### Домашнее задание к уроку №9

In [6]:
import pandas as pd
import numpy as np
import dill
from flask import Flask, request, jsonify
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier


##### Для работы был взят датасет содержащий данные по оттоку клиентов. Сама модель предсказаний выполнялась на 5 уроке.

In [10]:
path = 'c://users//andrei//downloads//lection9//'
df = pd.read_csv(path + 'churn_data.csv', sep=',', index_col='RowNumber')


In [11]:
df.tail()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [12]:
df.describe()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Разделим данные

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['CustomerId', 'Exited', 'Surname'], axis=1), df['Exited'], random_state=0)
X_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
RowNumber,,,,,,,,,,
2968,579,Germany,Female,39,5,117833.30,3,0,0,5831.00
701,750,France,Female,32,5,0.00,2,1,0,95611.47
3482,729,Spain,Female,34,9,53299.96,2,1,1,42855.97
1622,689,Spain,Male,38,5,75075.14,1,1,1,8651.92
801,605,France,Male,52,7,0.00,2,1,1,173952.50


Сохраним тестовые данные для проверки на стороне "клиента"

In [68]:
X_test.to_csv(path + 'x_test.csv')
y_test.to_csv(path + 'y_test.csv', index=None)

Строим Pipline

In [14]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [15]:
categorical_columns = ['Geography', 'Gender']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'Tenure']
not_changed_columns = ['HasCrCard', 'IsActiveMember']

In [16]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))
    
for nc_col in not_changed_columns:
    nc_transformer = Pipeline([
                    ('selector', NumberSelector(key=nc_col))
    ])
    final_transformers.append((nc_col, nc_transformer))

In [17]:
feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])
pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 55)),
])

Обучаем модель

In [19]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('CreditScore',
                                                 Pipeline(steps=[('selector',
      

Сохраняем модель

In [20]:
with open(path + "grad_bust_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [28]:
with open(path + "grad_bust_pipeline.dill", 'rb') as in_strm:
    model = dill.load(in_strm)

Проверяем работу после загрузки

In [69]:
predictions = model.predict_proba(X_test)
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test)


0.8750832577640639

In [23]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions[:,1])
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                                       fscore[ix],
                                                                                       precision[ix],
                                                                                       recall[ix]))

Best Threshold=0.329990, F-Score=0.646, Precision=0.632, Recall=0.660


Тестовый запуск сервера

In [9]:
app = Flask(__name__)
@app.route('/a')
def hello():
    return 'Hello world!'

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2023 17:25:14] "GET /a HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 17:25:14] "GET /favicon.ico HTTP/1.1" 404 -


Запуск демо-сервера.

In [66]:
app = Flask(__name__)
@app.route('/', methods=['GET'])
def start():
    return 'Welcome to churn`s prediction!'

@app.route('/predict', methods=['POST'])
def predict():
    data = {'Success': False}
    model_data = {
        'CreditScore': [],
        'Geography': [],
        'Gender': [],
        'Age': [],
        'Tenure': [],
        'Balance': [],
        'NumOfProducts': [],
        'HasCrCard': [],
        'IsActiveMember': [],
        'EstimatedSalary': [],
    }
    
    request_json = request.get_json()
    
    for key, value in model_data.items():
        if request_json[key]:
            model_data[key].append(request_json[key])
        else:
            model_data[key].append(0)
  
     
    prediction = model.predict_proba(pd.DataFrame(model_data))
    
    data['Prediction'] = prediction[:, 1][0]
    data['Success'] = True
    print('OK')
    
    return jsonify(data)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2023 08:50:24] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 08:50:24] "POST /predict HTTP/1.1" 200 -


{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 597, 'Geography': 'Germany', 'Gender': 'Female', 'Age': 35, 'Tenure': 8, 'Balance': 131101.04, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 192852.67}
filled dict {'CreditScore': [597], 'Geography': ['Germany'], 'Gender': ['Female'], 'Age': [35], 'Tenure': [8], 'Balance': [131101.04], 'NumOfProducts': [1], 'HasCrCard': [1], 'IsActiveMember': [1], 'EstimatedSalary': [192852.67]}
OK
{'Success': True, 'Prediction': 0.3032187329925324}
{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 523, 'Geography': 'France', 'Gender': 'Female', 'Age': 40, 'Tenure': 2, 'Balance': 102967.41, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 0, 'E

127.0.0.1 - - [27/May/2023 08:51:21] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 08:51:22] "POST /predict HTTP/1.1" 200 -


{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 597, 'Geography': 'Germany', 'Gender': 'Female', 'Age': 35, 'Tenure': 8, 'Balance': 131101.04, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 192852.67}
filled dict {'CreditScore': [597], 'Geography': ['Germany'], 'Gender': ['Female'], 'Age': [35], 'Tenure': [8], 'Balance': [131101.04], 'NumOfProducts': [1], 'HasCrCard': [1], 'IsActiveMember': [1], 'EstimatedSalary': [192852.67]}
OK
{'Success': True, 'Prediction': 0.3032187329925324}
{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 523, 'Geography': 'France', 'Gender': 'Female', 'Age': 40, 'Tenure': 2, 'Balance': 102967.41, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 0, 'E

...


{'CreditScore': 699, 'Geography': 'France', 'Gender': 'Male', 'Age': 64, 'Tenure': 9, 'Balance': 113109.52, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 0, 'EstimatedSalary': 27980.8}
filled dict {'CreditScore': [699], 'Geography': ['France'], 'Gender': ['Male'], 'Age': [64], 'Tenure': [9], 'Balance': [113109.52], 'NumOfProducts': [1], 'HasCrCard': [1], 'IsActiveMember': [0], 'EstimatedSalary': [27980.8]}
OK
{'Success': True, 'Prediction': 0.7794580261909552}
{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 560, 'Geography': 'Spain', 'Gender': 'Female', 'Age': 31, 'Tenure': 5, 'Balance': 125341.69, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 0, 'EstimatedSalary': 79547.39}
filled dict {'CreditScore': [560], 'Geography': ['Spain'], 'Gender': ['Female'], 'Age': [31], 'Tenure': [5], 'Balance': [125341.69], 'NumOfProducts': [1], 'H

127.0.0.1 - - [27/May/2023 08:53:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 08:53:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 08:53:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 08:53:47] "POST /predict HTTP/1.1" 200 -


OK
{'Success': True, 'Prediction': 0.12240461146695264}
{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 837, 'Geography': 'Germany', 'Gender': 'Male', 'Age': 38, 'Tenure': 2, 'Balance': 126732.85, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 79577.38}
filled dict {'CreditScore': [837], 'Geography': ['Germany'], 'Gender': ['Male'], 'Age': [38], 'Tenure': [2], 'Balance': [126732.85], 'NumOfProducts': [1], 'HasCrCard': [1], 'IsActiveMember': [1], 'EstimatedSalary': [79577.38]}
OK
{'Success': True, 'Prediction': 0.23024081433422336}
{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 773, 'Geography': 'Germany', 'Gender': 'Male', 'Age': 51, 'Tenure': 8, 'Balance': 116197.65, 'NumOfPro

127.0.0.1 - - [27/May/2023 08:53:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 08:53:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 08:53:47] "POST /predict HTTP/1.1" 200 -


{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 840, 'Geography': 'France', 'Gender': 'Female', 'Age': 39, 'Tenure': 1, 'Balance': 94968.97, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 0, 'EstimatedSalary': 84487.62}
filled dict {'CreditScore': [840], 'Geography': ['France'], 'Gender': ['Female'], 'Age': [39], 'Tenure': [1], 'Balance': [94968.97], 'NumOfProducts': [1], 'HasCrCard': [1], 'IsActiveMember': [0], 'EstimatedSalary': [84487.62]}
OK
{'Success': True, 'Prediction': 0.23574893590711732}
{'CreditScore': [], 'Geography': [], 'Gender': [], 'Age': [], 'Tenure': [], 'Balance': [], 'NumOfProducts': [], 'HasCrCard': [], 'IsActiveMember': [], 'EstimatedSalary': []}
{'CreditScore': 679, 'Geography': 'Germany', 'Gender': 'Male', 'Age': 38, 'Tenure': 7, 'Balance': 110555.37, 'NumOfProducts': 2, 'HasCrCard': 1, 'IsActiveMember': 0, 'Estimat